In [ ]:
import numpy as np
import numpy.ma as ma
import cv2
from matplotlib import pyplot as plt

from skimage import filters

In [ ]:
raw_data_dir = "raw_data/"

In [ ]:
def show_color_im(img):
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
img = cv2.imread(raw_data_dir + '2p22_00.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

thresh_val = np.median(gray)/1.15
ret, thresh = cv2.threshold(gray,thresh_val,255,cv2.THRESH_BINARY_INV)
plt.imshow(thresh)
plt.show()

In [ ]:
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
plt.imshow(opening)
plt.show()

In [ ]:
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.5*dist_transform.max(),255,0)
plt.imshow(sure_fg)
plt.show()

In [ ]:
mask = cv2.cvtColor(sure_fg,cv2.COLOR_GRAY2RGB)
img_masked = img.copy()
img_masked[mask==0] = 0
show_color_im(img_masked)
plt.show()
show_color_im(img)
plt.show()